In [123]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import importlib
import lumped_parameter_model
import utils 
importlib.reload(lumped_parameter_model)
importlib.reload(utils)
from lumped_parameter_model import *
from utils import *
from scipy import optimize
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import xgboost as xgb

### Features for model
- Precip
- Recharge
- Rolling average head
- Rolling avg precip
- Previous discharge ? 
- Effective K (use the K vs. head curve and estimate K)

In [4]:
dis = pd.read_csv('BSdischarge2017_2020.csv',header = 3)[['Date / Time', 
       '55A0000406 Adjusted Level (ft.)', 
      '55A0000406 Discharge (m^3/sec)']].dropna()
dis = dis.rename({'Date / Time' : "date", 
       '55A0000406 Adjusted Level (ft.)' : "water level[ft]", 
      '55A0000406 Discharge (m^3/sec)' : 'Q [m3/s]'}, axis = 1)
dis.to_csv("discharge_2017_2020.csv", index = False )

C:\Users\huan1428\AppData\Local\Temp\ipykernel_4104\2145101116.py:1: DtypeWarning: Columns (3,4,7,8,9,10,11,12,13,14,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  dis = pd.read_csv('BSdischarge2017_2020.csv',header = 3)[['Date / Time',


In [82]:
p1 = pd.read_csv('../precipitation/precip_2017_2021.csv')

C:\Users\huan1428\AppData\Local\Temp\ipykernel_33064\1573152887.py:1: DtypeWarning: Columns (42,43,44,50,52,57,76,90) have mixed types. Specify dtype option on import or set low_memory=False.
  p1 = pd.read_csv('../precipitation/precip_2017_2021.csv')


In [95]:
p = p1[['DATE', 'DailyPrecipitation']].rename({"DailyPrecipitation" : "PRCP"}, axis = 1)
p['PRCP'] = p['PRCP'].replace('T', 0).fillna(0).astype(float)

In [ ]:
p = convert_df_to_date_time(p, 'DATE')
p.index = p['DATE']


In [97]:
p = p.groupby(p.index.date).max()
p.to_csv("../precipitation/precip_clean_2017_2021.csv", index = False)

In [98]:
dis = convert_df_to_date_time(pd.read_csv('discharge_2017_2020.csv'), 'date').rename({"date" : "DATE"}, axis = 1)[['DATE', 'Q [m3/s]']]
precip = convert_df_to_date_time(pd.read_csv('../precipitation/precip_clean_2017_2021.csv'), "DATE")
rech = convert_df_to_date_time(pd.read_csv('2017_2021_simulated_recharge_hourly.csv'), 'DATE')
h = convert_df_to_date_time(pd.read_csv('../gw_level/gw_2018_2021_clean.csv'), 'date').rename({"date" : "DATE"}, axis = 1)

c:\Users\huan1428\Documents\karst\lumped\utils.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])


In [99]:
daily_Q = convert_hourly_to_daily(dis, 'DATE')
daily_r = convert_hourly_to_daily(rech, 'DATE')

In [101]:
daily_Q.index = daily_Q['DATE']
daily_r.index = daily_r['DATE']
precip.index = precip['DATE'].dt.date
h.index = h['DATE']
daily_Q = daily_Q.drop("DATE", axis = 1)
daily_r = daily_r.drop("DATE", axis = 1)
precip = precip.drop("DATE", axis = 1)
h = h.drop("DATE", axis = 1)

In [107]:
df = daily_Q.join(h).join(precip).join(daily_r)
df.to_csv('ml_daily_raw_data.csv')

In [77]:
hourly_p = convert_daily_to_hourly(precip[['DATE', 'PRCP']], 'DATE', interp_method= 'uniform')
hourly_h = convert_daily_to_hourly(h[['DATE', 'gw_elev[ft]']], 'DATE')

In [79]:
#full dataset
df = pd.merge(pd.merge(pd.merge(hourly_h, hourly_p, how = 'outer', on = 'DATE'),rech, how = 'right', on = 'DATE'), dis, how = 'left', on = 'DATE')
df.to_csv('ml_raw_hourly_data.csv', index = False)

In [ ]:
df = pd.read_csv('ml_daily_raw_data.csv')

,DATE,Q [m3/s],depth to water level[ft],gw_elev[ft],PRCP,RCH[m3/s],RCH[m/day]
0,2017-02-02,0.024998,4.74,1249.76,0.0,0.034298,0.000640
1,2017-02-03,0.024998,4.98,1249.52,0.0,0.033036,0.000616
2,2017-02-04,0.024998,5.20,1249.30,0.0,0.031798,0.000593
3,2017-02-05,0.023769,5.49,1249.01,0.0,0.031574,0.000589
4,2017-02-06,0.023769,5.73,1248.77,0.0,0.030795,0.000575
...,...,...,...,...,...,...,...
1298,2020-08-23,0.068539,13.26,1241.24,0.0,0.028748,0.000536
1299,2020-08-24,0.070828,13.34,1241.16,0.0,0.028192,0.000526
1300,2020-08-25,0.070828,13.41,1241.09,0.0,0.027656,0.000516
1301,2020-08-26,0.070828,13.49,1241.01,0.0,0.026578,0.000496


In [112]:
df = df[df['Q [m3/s]'].isna() == False] #filter out empty Q values
df['gw_elev[m]'] = df['gw_elev[ft]'] * .3048


In [ ]:
try:
    df.index = pd.to_datetime(df['DATE']) #set index to date 
except: 
    print("index not set")
    df = df.drop('DATE', axis = 1)

In [114]:
#time series related features
df['month'] = df.index.month
df['is_growing_season'] = df['month'].isin([6,7,8,9]).astype(int)

In [116]:
#rolling average Q
df['Q 1d avg'] =  df['Q [m3/s]'].shift(1).rolling(window ='1D').mean()
df['Q 3d avg'] = df['Q [m3/s]'].shift(1).rolling(window ='3D').mean()
df['Q 7d avg']= df['Q [m3/s]'].shift(1).rolling(window ='7D').mean()
# df['Q 12h avg'] = df['Q [m3/s]'].shift(1).rolling(window ='12h').mean()

In [117]:
#precip
df['precip cum 1d [in]'] = df['PRCP'].rolling(window = '1D').sum()
df['precip cum 3d [in]'] = df['PRCP'].rolling(window = '3D').sum()
df['precip cum 7d [in]'] = df['PRCP'].rolling(window = '7D').sum()


In [118]:
#head 
df['head 1d avg'] = df['gw_elev[m]'].shift(1).rolling(window = '1D').mean()
df['head 3d avg'] = df['gw_elev[m]'].shift(1).rolling(window = '3D').mean()
df['head 7d avg'] = df['gw_elev[m]'].shift(1).rolling(window = '7D').mean()

In [119]:
df

,Q [m3/s],depth to water level[ft],gw_elev[ft],PRCP,RCH[m3/s],RCH[m/day],gw_elev[m],month,is_growing_season,Q 1d avg,Q 3d avg,Q 7d avg,Q 12h avg,precip cum 1d [in],precip cum 3d [in],precip cum 7d [in],head 1d avg,head 3d avg,head 7d avg
DATE,,,,,,,,,,,,,,,,,,,
2017-02-02,0.024998,4.74,1249.76,0.0,0.034298,0.000640,380.926848,2,0,NaN,NaN,NaN,NaN,0.0,0.00,0.00,NaN,NaN,NaN
2017-02-03,0.024998,4.98,1249.52,0.0,0.033036,0.000616,380.853696,2,0,0.024998,0.024998,0.024998,0.024998,0.0,0.00,0.00,380.926848,380.926848,380.926848
2017-02-04,0.024998,5.20,1249.30,0.0,0.031798,0.000593,380.786640,2,0,0.024998,0.024998,0.024998,0.024998,0.0,0.00,0.00,380.853696,380.890272,380.890272
2017-02-05,0.023769,5.49,1249.01,0.0,0.031574,0.000589,380.698248,2,0,0.024998,0.024998,0.024998,0.024998,0.0,0.00,0.00,380.786640,380.855728,380.855728
2017-02-06,0.023769,5.73,1248.77,0.0,0.030795,0.000575,380.625096,2,0,0.023769,0.024588,0.024691,0.023769,0.0,0.00,0.00,380.698248,380.779528,380.816358
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-08-23,0.068539,13.26,1241.24,0.0,0.028748,0.000536,378.329952,8,1,0.068539,0.068539,0.069193,0.068539,0.0,0.39,0.39,378.345192,378.366528,378.424440
2020-08-24,0.070828,13.34,1241.16,0.0,0.028192,0.000526,378.305568,8,1,0.068539,0.068539,0.068866,0.068539,0.0,0.33,0.39,378.329952,378.346208,378.398314
2020-08-25,0.070828,13.41,1241.09,0.0,0.027656,0.000516,378.284232,8,1,0.070828,0.069302,0.068866,0.070828,0.0,0.00,0.39,378.305568,378.326904,378.373059
